In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append('../')

In [ ]:
# specify parameters for interactive run
# this cell will be replaced during pipeline run with the parameters from notebook function call
bento_service = '/tmp/step/house_price-lite/exp-pipeline/run-24-12-27-072817/bento_service'
substep_params={
    "rmse_threshold": 15
}

In [ ]:
# Test model API
from sinara.bentoml import start_dev_bentoservice, stop_dev_bentoservice, load_bentoservice
from sklearn.metrics import root_mean_squared_error
import pandas as pd
import requests
import json

# Load BentoService
bento_serv = load_bentoservice(bento_service)

# Stop a dev model server if running
stop_dev_bentoservice(bento_serv)

# Start a dev model server to test out the API endpoint locally
start_dev_bentoservice(bento_serv)

serv_v = json.loads(requests.post("http://127.0.0.1:5000/service_version", json={}).text)
print(serv_v)

#print(requests.post("http://127.0.0.1:5000/test_data", json={}).text)
test_data = json.loads(requests.post("http://127.0.0.1:5000/test_data", json={}).text)

preds = json.loads(requests.post("http://127.0.0.1:5000/predict", json=test_data['X']).text)

rmse = root_mean_squared_error(pd.DataFrame(test_data['Y']).values, preds)
print("The root mean squared error (RMSE) on eval set: {:.4f}".format(rmse))

# Stop the dev model server
stop_dev_bentoservice(bento_serv)

In [ ]:
# check eval result
rmse_threshold = substep_params["rmse_threshold"]
if rmse > rmse_threshold:
    raise Exception(f'RMSE is {rmse}, more than acceptable value of {rmse_threshold}')